In [1]:
import cv2 as cv
import numpy as np
import os

## Import Images

In [6]:
path = '../../Resources/Images/FeatureExtraction/Images/'
images = []
itemNames = []

myList = os.listdir(path)
for item in myList:
    imgCur = cv.imread(f'{path}{item}', 0)
    images.append(imgCur)
    itemNames.append(os.path.splitext(item)[0])

print("Images: ", len(images))
print("Image Names : ", itemNames)

Images:  6
Image Names :  ['DSA', 'Data Structure book', 'Image Processing book', 'vision_book', 'vision_book_original', 'vision book']


## Create Descriptors for the Imported Images

In [3]:
orb = cv.ORB_create(nfeatures=1000)

def findDes(images):
    desList = []
    for img in images:
        kp1, des1 = orb.detectAndCompute(img, None)
        desList.append(des1)
    return desList

desList = findDes(images)
print("Descriptors : ", len(desList))

Descriptors :  6


## Descriptor Match
#### Create descriptor for the search image and match with the descriptor of imported images

In [4]:
def findID(img, desList, thresh = 15):
    kp2, des2 = orb.detectAndCompute(img, None)

    # Matching
    bf = cv.BFMatcher()
    matchList = []
    ID = -1
    try:
        for des in desList:
            matches = bf.knnMatch(des, des2, k = 2)
            # Check Good Match
            good =[]
            for m, n in matches:
                if m.distance < 0.75 * n.distance:
                    good.append([m])
            matchList.append(len(good))
    except:
        pass
    # print(matchList)
    if len(matchList) != 0:
        if max(matchList) > thresh:
            ID = matchList.index(max(matchList))
    return ID

## Search from secondary camera

In [5]:
cap = cv.VideoCapture(0)
while True:
    isTrue, frame = cap.read()
    imgOG = frame.copy()
    imgGray = cv.cvtColor(imgOG, cv.COLOR_BGR2GRAY)
    

    ID = findID(imgGray, desList)
    if ID != -1:
        cv.putText(imgOG, itemNames[ID], (50,50), cv.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
    cv.imshow(" Image 2 ", imgOG)
    if cv.waitKey(1) & 0xFF == ord('x'):
        break

cap.release()
cv.destroyAllWindows()